In [1]:
import os

go_up_n_directories = lambda path, n: os.path.abspath(
    os.path.join(*([os.path.dirname(path)] + [".."] * n))
)
os.chdir(go_up_n_directories(os.getcwd(), 4))  # run once (otherwise restart kernel)
os.getcwd()

'/home/tim/Development/OCPPM'

In [2]:
# Python natives
import pickle
import re
import time
import os
import json
import pprint

# Data handling
from ocpa.algo.predictive_monitoring.obj import Feature_Storage
from ocpa.algo.predictive_monitoring import tabular

# Booster model
import lightgbm as lgb

import utilities.evaluation_utils as evaluation_utils

# Config
feature_storage_file = "data/BPI17/feature_encodings/EFG/efg/raw/BPI_split_[C2_P2_P3_P5_O3_Action_EventOrigin_OrgResource].fs"
target = "event_remaining_time"
model_output_path = "models/BPI17/baselines/eft"

In [3]:
with open(feature_storage_file, "rb") as bin:
    feature_storage: Feature_Storage = pickle.load(bin)

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
# flatten EFG with same train/test split
eft_train = tabular.construct_table(
    feature_storage, feature_storage.train_indices + feature_storage.validation_indices
)
eft_test = tabular.construct_table(feature_storage, feature_storage.test_indices)

# rename columns that contain JSON special characters (as they are not supported by LightGBM)
rename_columns = lambda col_name: re.sub("[^A-Za-z0-9_]+", "", str(col_name))
eft_train = eft_train.rename(columns=rename_columns)
eft_test = eft_test.rename(columns=rename_columns)

In [5]:
# import dataset into LightGBM framework
X_train, y_train = (
    eft_train.drop(columns=[target]),
    eft_train.loc[:, target],
)
X_valid, y_valid = (
    eft_test.drop(columns=[target]),
    eft_test.loc[:, target],
)

train_data = lgb.Dataset(
    X_train,
    label=y_train,
)
valid_data = lgb.Dataset(
    X_valid,
    label=y_valid,
)

In [6]:
params = {
    "objective": "regression",
    "metric": ["mse", "mae", "mape", "rmse"],
    "num_boost_round": 100,
    "stopping_rounds": 100,
}

start_train_time = time.time()
bst = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(params["stopping_rounds"])],
)
total_train_time = time.time() - start_train_time

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.232638 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 752
[LightGBM] [Info] Number of data points in the train set: 276187, number of used features: 26
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Info] Start training from score 0.001795
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[98]	valid_0's l2: 0.566428	valid_0's l1: 0.528217	valid_0's mape: 0.446807	valid_0's rmse: 0.752614


In [7]:
pred_start_time = time.time()
y_train_preds = bst.predict(X_train)
train_pred_time = time.time() - pred_start_time

pred_start_time = time.time()
y_valid_preds = bst.predict(X_valid)
valid_pred_time = time.time() - pred_start_time

In [8]:
eval_train = evaluation_utils.get_evaluation(
    y_train, y_train_preds, regression=True, time=train_pred_time
)
eval_train["report"]["training_time"] = total_train_time
eval_valid = evaluation_utils.get_evaluation(
    y_valid, y_valid_preds, regression=True, time=valid_pred_time
)

experiment_settings = {"experiment_settings": params}
evaluation_report = {"Train": eval_train, "Test": eval_valid}

In [9]:
with open(os.path.join(model_output_path, "experiment_settings.json"), "w") as fp:
    json.dump(experiment_settings, fp, indent=2)
with open(os.path.join(model_output_path, "evaluation_report.json"), "w") as fp:
    json.dump(evaluation_report, fp, indent=2)

pprint.pprint(evaluation_report)

{'Test': {'report': {'MAE': 0.528216573078087,
                     'MAPE': 8.21843932478464,
                     'MSE': 0.5664282332331936,
                     'R^2': 0.42133709021710364,
                     'prediction_time': 0.2623112201690674}},
 'Train': {'report': {'MAE': 0.5282096004108215,
                      'MAPE': 8.385424154947259,
                      'MSE': 0.5730202952019726,
                      'R^2': 0.42978440206475876,
                      'prediction_time': 0.5728161334991455,
                      'training_time': 15.922951459884644}}}
